# 법률 질의응답 AI 시스템 (Legal QA with RAG)

---

## 프로젝트 개요

**목표**: 여러 법령 PDF 파일을 기반으로 법률적 질문에 대한 정확한 답변을 제공하는 RAG 시스템 구축

**주요 기능**:
1. 📄 여러 PDF 법령 문서 로드 및 처리
2. ✂️ 법률 조문에 최적화된 텍스트 분할
3. 🔢 고품질 임베딩 생성 (한국어 최적화)
4. 🗄️ 벡터 데이터베이스 구축 및 검색
5. 🤖 법률 전문 프롬프트 기반 답변 생성
6. 📊 출처 및 근거 조문 명시

---

## 1. 환경 설정

In [ ]:
# 환경 변수 로드
from dotenv import load_dotenv
load_dotenv()

In [ ]:
# 기본 라이브러리
import os
from glob import glob
from pprint import pprint
import warnings
warnings.filterwarnings('ignore')

## 2. PDF 문서 로드

법령 PDF 파일들을 로드합니다.

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import DirectoryLoader

# 데이터 디렉토리 경로
DATA_DIR = './data'

# PDF 파일 목록 확인
pdf_files = glob(os.path.join(DATA_DIR, '*.pdf'))
print(f"발견된 PDF 파일 수: {len(pdf_files)}")
print("\n파일 목록:")
for pdf_file in pdf_files:
    print(f"  - {os.path.basename(pdf_file)}")

In [ ]:
# 법령 PDF 파일만 선택 (필요시 수정)
# 여기서는 housing_leasing_law.pdf를 사용
legal_pdf_files = [
    os.path.join(DATA_DIR, 'housing_leasing_law.pdf'),
    # 추가 법령 PDF 파일이 있다면 여기에 추가
]

# 문서 로드
all_documents = []

for pdf_file in legal_pdf_files:
    if os.path.exists(pdf_file):
        print(f"\n로딩 중: {os.path.basename(pdf_file)}")
        loader = PyPDFLoader(pdf_file)
        documents = loader.load()
        all_documents.extend(documents)
        print(f"  ✓ {len(documents)} 페이지 로드 완료")
    else:
        print(f"  ✗ 파일을 찾을 수 없습니다: {pdf_file}")

print(f"\n총 로드된 페이지 수: {len(all_documents)}")

In [ ]:
# 첫 번째 문서 확인
if all_documents:
    print("첫 번째 페이지 미리보기:")
    print("=" * 100)
    print(all_documents[0].page_content[:500])
    print("=" * 100)
    print(f"\n메타데이터: {all_documents[0].metadata}")

## 3. 텍스트 분할 (법률 조문 최적화)

법률 문서는 조문 단위로 구조화되어 있으므로, 이를 고려한 분할 전략을 사용합니다.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 법률 문서에 최적화된 텍스트 분할기
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,              # 법률 조문을 포함할 수 있는 적절한 크기
    chunk_overlap=200,           # 문맥 보존을 위한 중복
    length_function=len,
    separators=[
        "\n\n",                   # 단락 구분
        "\n",                     # 줄 구분
        "제",                     # 조문 구분 (예: 제1조, 제2조)
        "。",                     # 문장 구분
        ". ",                    # 문장 구분
        " ",                     # 단어 구분
        "",
    ],
    is_separator_regex=False,
)

# 문서 분할
split_documents = text_splitter.split_documents(all_documents)

print(f"분할된 청크 수: {len(split_documents)}")
print(f"평균 청크 크기: {sum(len(doc.page_content) for doc in split_documents) / len(split_documents):.0f} 문자")

In [ ]:
# 분할된 청크 샘플 확인
print("분할된 청크 샘플 (처음 3개):")
print("=" * 100)
for i, doc in enumerate(split_documents[:3], 1):
    print(f"\n[청크 {i}]")
    print(f"길이: {len(doc.page_content)} 문자")
    print(f"내용: {doc.page_content[:300]}...")
    print("-" * 100)

## 4. 임베딩 모델 설정

한국어 법률 문서에 최적화된 임베딩 모델을 사용합니다.

In [ ]:
from langchain_openai import OpenAIEmbeddings

# OpenAI 임베딩 모델 (한국어 성능 우수)
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",  # 비용 효율적이면서 성능 좋음
    # model="text-embedding-3-large",  # 더 높은 성능이 필요한 경우
)

print("✓ 임베딩 모델 초기화 완료")
print(f"  모델: {embeddings.model}")
print(f"  차원: {embeddings.dimensions if hasattr(embeddings, 'dimensions') and embeddings.dimensions else '1536 (기본값)'}")

## 5. 벡터 데이터베이스 구축

Chroma DB를 사용하여 벡터 저장소를 구축합니다.

In [ ]:
from langchain_chroma import Chroma

# 벡터 저장소 디렉토리
VECTOR_DB_DIR = "./chroma_db_legal"

# Chroma 벡터 저장소 생성
print("벡터 데이터베이스 생성 중...")
print("(임베딩 생성에 시간이 걸릴 수 있습니다)\n")

vector_store = Chroma.from_documents(
    documents=split_documents,
    embedding=embeddings,
    collection_name="legal_documents",
    persist_directory=VECTOR_DB_DIR,
    collection_metadata={"hnsw:space": "cosine"},  # 코사인 유사도 사용
)

print(f"✓ 벡터 데이터베이스 생성 완료")
print(f"  저장된 문서 수: {len(vector_store.get()['ids'])}")
print(f"  저장 위치: {VECTOR_DB_DIR}")

## 6. 검색기(Retriever) 설정

다양한 검색 전략을 테스트합니다.

In [ ]:
# 기본 유사도 검색
retriever_similarity = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}  # 상위 5개 문서 검색
)

# MMR 검색 (다양성 고려)
retriever_mmr = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 5,
        "fetch_k": 20,
        "lambda_mult": 0.7,  # 0.7 = 관련성과 다양성의 균형
    }
)

print("✓ 검색기 설정 완료")
print("  - 유사도 검색 (retriever_similarity)")
print("  - MMR 검색 (retriever_mmr)")

In [ ]:
# 검색 테스트
test_query = "임대차 계약 기간은 어떻게 되나요?"

print(f"테스트 질문: {test_query}\n")
print("=" * 100)

retrieved_docs = retriever_mmr.invoke(test_query)

print(f"검색된 문서 수: {len(retrieved_docs)}\n")

for i, doc in enumerate(retrieved_docs, 1):
    print(f"[문서 {i}]")
    print(f"출처: {doc.metadata.get('source', 'Unknown')} (페이지 {doc.metadata.get('page', 'N/A')})")
    print(f"내용: {doc.page_content[:200]}...")
    print("-" * 100)

## 7. 법률 전문 프롬프트 설계

법률 질의응답에 특화된 프롬프트를 작성합니다.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# 법률 전문 시스템 프롬프트
system_prompt = """당신은 대한민국 법률 전문 AI 어시스턴트입니다.

<역할 및 책임>
- 제공된 법령 조문을 기반으로 정확하고 신뢰할 수 있는 법률 정보를 제공합니다.
- 법률 용어를 일반인도 이해하기 쉽게 설명합니다.
- 답변의 근거가 되는 법령 조문을 반드시 명시합니다.

<답변 원칙>
1. **정확성**: 제공된 컨텍스트에 있는 내용만을 기반으로 답변합니다.
2. **명확성**: 법률 용어를 쉽게 풀어서 설명합니다.
3. **근거 제시**: 답변의 근거가 되는 조문을 명확히 밝힙니다.
4. **한계 인정**: 컨텍스트에 정보가 없으면 "제공된 법령에서 해당 내용을 찾을 수 없습니다"라고 답변합니다.
5. **주의사항**: 법률 자문이 아님을 명시하고, 구체적인 사안은 전문가 상담을 권장합니다.

<답변 형식>
1. **핵심 답변**: 질문에 대한 직접적인 답변
2. **법적 근거**: 관련 법령 조문 인용 (조문 번호 포함)
3. **추가 설명**: 필요시 보충 설명
4. **주의사항**: 법률 자문이 아님을 명시

답변은 반드시 한국어로 작성하세요.
"""

user_prompt = """<제공된 법령 조문>
{context}
</제공된 법령 조문>

<질문>
{question}
</질문>

위 법령 조문을 바탕으로 질문에 답변해주세요.
"""

# 프롬프트 템플릿 생성
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", user_prompt)
])

print("✓ 법률 전문 프롬프트 템플릿 생성 완료")

## 8. RAG 체인 구성

전체 RAG 파이프라인을 구축합니다.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

# LLM 설정
llm = ChatOpenAI(
    model="gpt-4o-mini",  # 또는 "gpt-4o" (더 높은 성능)
    temperature=0.1,       # 낮은 temperature로 일관성 있는 답변
)

# 문서 포맷팅 함수
def format_docs(docs):
    """검색된 문서를 프롬프트에 맞게 포맷팅"""
    formatted = []
    for i, doc in enumerate(docs, 1):
        source = doc.metadata.get('source', 'Unknown')
        page = doc.metadata.get('page', 'N/A')
        formatted.append(
            f"[문서 {i}] (출처: {os.path.basename(source)}, 페이지: {page})\n"
            f"{doc.page_content}\n"
        )
    return "\n---\n".join(formatted)

# RAG 체인 생성
rag_chain = (
    RunnableParallel(
        {
            "context": retriever_mmr | format_docs,
            "question": RunnablePassthrough()
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

print("✓ RAG 체인 구성 완료")

## 9. 법률 질의응답 테스트

실제 법률 질문으로 시스템을 테스트합니다.

In [ ]:
# 테스트 질문들
test_questions = [
    "주택 임대차 계약의 최소 기간은 얼마인가요?",
    "전세금 반환은 언제 받을 수 있나요?",
    "임대인이 계약 갱신을 거부할 수 있는 경우는 언제인가요?",
    "임차인의 우선변제권이란 무엇인가요?",
]

print("법률 질의응답 테스트\n")
print("=" * 100)

for i, question in enumerate(test_questions, 1):
    print(f"\n[질문 {i}] {question}\n")
    
    # RAG 체인 실행
    answer = rag_chain.invoke(question)
    
    print(f"[답변]\n{answer}")
    print("\n" + "=" * 100)

## 10. 대화형 인터페이스

사용자가 직접 질문할 수 있는 인터페이스를 제공합니다.

In [ ]:
def ask_legal_question(question: str) -> str:
    """법률 질문에 답변하는 함수"""
    print(f"\n질문: {question}\n")
    print("답변 생성 중...\n")
    
    answer = rag_chain.invoke(question)
    
    print("=" * 100)
    print(answer)
    print("=" * 100)
    
    return answer

# 사용 예시
# ask_legal_question("임대차 계약 기간 중 임대료를 인상할 수 있나요?")

In [ ]:
# 여기에 질문을 입력하세요
my_question = "임대차 계약 기간 중 임대료를 인상할 수 있나요?"

answer = ask_legal_question(my_question)

## 11. 검색 결과 분석

어떤 문서가 검색되었는지 확인합니다.

In [ ]:
def analyze_retrieval(question: str):
    """검색 결과를 상세히 분석"""
    print(f"질문: {question}\n")
    print("=" * 100)
    
    # 검색 실행
    retrieved_docs = retriever_mmr.invoke(question)
    
    print(f"\n검색된 문서 수: {len(retrieved_docs)}\n")
    
    for i, doc in enumerate(retrieved_docs, 1):
        print(f"\n[문서 {i}]")
        print(f"출처: {doc.metadata.get('source', 'Unknown')}")
        print(f"페이지: {doc.metadata.get('page', 'N/A')}")
        print(f"문서 길이: {len(doc.page_content)} 문자")
        print(f"\n내용:\n{doc.page_content}")
        print("-" * 100)

# 사용 예시
# analyze_retrieval("전세금 반환은 언제 받을 수 있나요?")

## 12. 추가 PDF 파일 추가하기

새로운 법령 PDF를 추가하는 방법

In [ ]:
def add_new_legal_documents(pdf_paths: list):
    """
    새로운 법령 PDF 파일을 벡터 데이터베이스에 추가
    
    Args:
        pdf_paths: 추가할 PDF 파일 경로 리스트
    """
    print("새로운 문서 추가 중...\n")
    
    # 문서 로드
    new_documents = []
    for pdf_path in pdf_paths:
        if os.path.exists(pdf_path):
            print(f"로딩: {os.path.basename(pdf_path)}")
            loader = PyPDFLoader(pdf_path)
            docs = loader.load()
            new_documents.extend(docs)
            print(f"  ✓ {len(docs)} 페이지 로드 완료")
        else:
            print(f"  ✗ 파일을 찾을 수 없습니다: {pdf_path}")
    
    if not new_documents:
        print("\n추가할 문서가 없습니다.")
        return
    
    # 텍스트 분할
    print(f"\n텍스트 분할 중...")
    split_docs = text_splitter.split_documents(new_documents)
    print(f"  ✓ {len(split_docs)} 청크 생성 완료")
    
    # 벡터 데이터베이스에 추가
    print(f"\n벡터 데이터베이스에 추가 중...")
    vector_store.add_documents(split_docs)
    print(f"  ✓ 추가 완료")
    
    print(f"\n현재 총 문서 수: {len(vector_store.get()['ids'])}")

# 사용 예시:
# new_pdfs = [
#     './data/새로운_법령1.pdf',
#     './data/새로운_법령2.pdf',
# ]
# add_new_legal_documents(new_pdfs)

## 13. 시스템 성능 개선 팁

### 더 나은 성능을 위한 방법:

1. **임베딩 모델 업그레이드**
   ```python
   # text-embedding-3-large 사용 (더 높은 정확도)
   embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
   ```

2. **LLM 모델 업그레이드**
   ```python
   # GPT-4o 사용 (더 정확한 법률 해석)
   llm = ChatOpenAI(model="gpt-4o", temperature=0.1)
   ```

3. **청크 크기 조정**
   - 법률 조문이 길면 `chunk_size`를 늘리기
   - 더 세밀한 검색이 필요하면 `chunk_size`를 줄이기

4. **검색 개수 조정**
   ```python
   # 더 많은 컨텍스트가 필요한 경우
   retriever = vector_store.as_retriever(
       search_type="mmr",
       search_kwargs={"k": 10}  # 10개 문서 검색
   )
   ```

5. **재순위화(Reranking) 추가**
   - 검색 결과를 재정렬하여 더 관련성 높은 문서 우선
   - Cohere Rerank API 또는 Cross-Encoder 사용

6. **쿼리 확장(Query Expansion)**
   - 사용자 질문을 여러 형태로 변환하여 검색
   - Multi-Query Retriever 사용

## 14. 프로젝트 저장 및 재사용

벡터 데이터베이스는 이미 `./chroma_db_legal`에 저장되어 있습니다.

### 다음번에 사용할 때:

```python
# 기존 벡터 데이터베이스 로드
vector_store = Chroma(
    persist_directory="./chroma_db_legal",
    embedding_function=embeddings,
    collection_name="legal_documents"
)

# 검색기 생성
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 5}
)

# RAG 체인 재구성
# (위의 코드 재사용)
```

## 15. 주의사항 및 면책조항

⚠️ **중요한 주의사항**:

1. **법률 자문 아님**: 이 시스템은 법률 정보 제공 목적이며, 공식적인 법률 자문이 아닙니다.

2. **전문가 상담 필요**: 구체적인 법률 문제는 반드시 변호사 등 법률 전문가와 상담하세요.

3. **최신성 확인**: 법령은 수시로 개정되므로, 최신 법령을 확인해야 합니다.

4. **정확성 검증**: AI의 답변은 참고용이며, 중요한 결정 전에는 원문을 직접 확인하세요.

5. **개인정보 보호**: 실제 사건이나 개인정보를 입력하지 마세요.